In [5]:
import numpy as np 
import xarray as xr
#from os.path import join 

#### Path variables

In [6]:
import os
os.getcwd()

'/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/5.-Data_Split_train_val_test/CERRA'

In [7]:
path = '/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/'
input_path = "3.-data_with_NAN_values_corrected/CERRA/"
data_file = "cleaned_aemet_daily-wind_m_s_2010-2022_CERRA.nc"
mask_file = "mask_aemet_daily-wind_2010-2022_CERRA.nc"

exp_name_in_data = "ws" # wind speed
exp_name_out_data = "ws" # wind speed
exp_name_in_mask = "mask" # wind speed
exp_name_out_mask = "ws" # wind speed

#### Split data by years

In [8]:
xr.open_dataset(path + input_path + mask_file)

<xarray.Dataset> Size: 1GB
Dimensions:  (time: 4656, y: 250, x: 250)
Coordinates:
  * time     (time) datetime64[ns] 37kB 2010-01-01 2010-01-02 ... 2022-09-30
  * y        (y) float64 2kB -1.727e+06 -1.722e+06 ... -3.63e+05 -3.575e+05
  * x        (x) float64 2kB -1.617e+06 -1.612e+06 ... -2.53e+05 -2.475e+05
Data variables:
    mask     (time, y, x) float32 1GB ...

In [ ]:
def split_data_byyears(mask_path, data_path, years, opath):

    print(data_path)

    dataset = xr.open_dataset(data_path)
    mask_dataset = xr.open_dataset(mask_path)
    #dataset = dataset.rename({exp_name_in_data: exp_name_out_data}) #needed in the kadow code
    mask_dataset = mask_dataset.rename({exp_name_in_mask: exp_name_out_mask}) #needed in the kadow code

    hdata = dataset['ws']
    hmask = mask_dataset['ws']

    print("Shape of the data: ", hdata.shape)
    print("Shape of the mask: ", hmask.shape)

    aemet_test_data = hdata.sel(time=slice(str(years[0]), str(years[1])))
    aemet_test_mask = hmask.sel(time=slice(str(years[0]), str(years[1])))
    
    print("")
    print("Shape of the aemet test data: ", aemet_test_data.shape)
    print("Shape of the aemet test mask: ", aemet_test_mask.shape)

    aemet_test_data.to_netcdf(opath + "aemet_daily-wind_m_s_" + str(years[0]) + "-" + str(years[1]) + "_CERRA.nc")
    aemet_test_mask.to_netcdf(opath + "mask_aemet_daily-wind_m_s_" + str(years[0]) + "-" + str(years[1]) + "_CERRA.nc")

    return hmask, hdata  

#### Main

In [10]:
if __name__ == '__main__':
    np.random.seed(42)

    #split_ratio = 7 / 9
    #split_test1 = 5 / 10
    #split_test2 = 2 / 3 

    years = [2010, 2022]

    output_path = "5.-Data_Split_train_val_test/CERRA/Split_data_" + str(years[0]) + "-" + str(years[-1]) + "/AEMET/"

    if not os.path.exists(path + output_path):
        os.makedirs(path + output_path)

    mask, data = split_data_byyears(path + input_path + mask_file, path + input_path + data_file, years, path + output_path)

/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/3.-data_with_NAN_values_corrected/CERRA/cleaned_aemet_daily-wind_m_s_2010-2022_CERRA.nc
Shape of the data:  (4656, 250, 250)
Shape of the mask:  (4656, 250, 250)

Shape of the aemet test data:  (4656, 250, 250)
Shape of the aemet test mask:  (4656, 250, 250)


In [14]:
data

<xarray.DataArray 'ws' (time: 4656, y: 250, x: 250)> Size: 1GB
[291000000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 37kB 2010-01-01 2010-01-02 ... 2022-09-30
  * y        (y) float64 2kB -1.727e+06 -1.722e+06 ... -3.63e+05 -3.575e+05
  * x        (x) float64 2kB -1.617e+06 -1.612e+06 ... -2.53e+05 -2.475e+05
Attributes:
    Dataset:  AEMET
    Version:  postclimatol-AEMET
    Creator:  NuriaPPlaza
    Date:     2025-04-09

#### Check the ratio of Non-zero AEMET data

In [16]:
print("Number of AEMET data: ", data.count().item())
print("Number of Non-zero AEMET data: ", (data > 0.).sum().item())
print("Ratio of zero AEMET data: {}", 100 - (data > 0.).sum().item() * 100 / data.count().item(), "%")
print("Ratio of Non-zero AEMET data: {}", (data > 0.).sum().item() * 100 / data.count().item(), "%")

Number of AEMET data:  291000000
Number of Non-zero AEMET data:  2692625
Ratio of zero AEMET data: {} 99.07469931271477 %
Ratio of Non-zero AEMET data: {} 0.9253006872852234 %


#### Check the ratio of Non-zero Mask data

In [17]:
print("Number of mask data: ", mask.count().item())
print("Number of Non-zero mask data: ", (mask > 0.).sum().item())
print("Ratio of zero mask data: ", 100 - (mask > 0.).sum().item() * 100 / data.count().item(), "%")
print("Ratio of Non-zero mask data: ", (mask > 0.).sum().item() * 100 / data.count().item(), "%")

Number of mask data:  291000000
Number of Non-zero mask data:  2705850
Ratio of zero mask data:  99.07015463917526 %
Ratio of Non-zero mask data:  0.9298453608247422 %
